<a href="https://colab.research.google.com/github/RoaaM/SemEval_competion/blob/main/Task_B(categorical_classification_4_classes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 36.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.0/266.0 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import ktrain
from ktrain import text
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/train_all_tasks.csv')
df = df[['text', 'label_category']]
df.head()

,text,label_category
0,"Damn, this writing was pretty chaotic",none
1,"Yeah, and apparently a bunch of misogynistic v...",none
2,How the FUCK is this woman still an MP!!!???,none
3,Understand. Know you're right. At same time I ...,none
4,Surprized they didn't stop and rape some women,none


In [ ]:
df['label_category'].value_counts()

none                                        10602
2. derogation                                1590
3. animosity                                 1165
4. prejudiced discussions                     333
1. threats, plans to harm and incitement      310
Name: label_category, dtype: int64

In [ ]:
df['label_category'] = df['label_category'].map({'1. threats, plans to harm and incitement':0,
                                                 '2. derogation':1,
                                                 '3. animosity':2,
                                                 '4. prejudiced discussions':3})

In [ ]:
df['label_category'].unique()

array([nan,  1.,  0.,  2.,  3.])

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df['label_category'] = df['label_category'].astype(int)

In [ ]:
df['label_category'].value_counts()

1    1590
2    1165
3     333
0     310
Name: label_category, dtype: int64

In [ ]:
df.shape

(3398, 2)

In [ ]:
X = df['text'].tolist()
y = df['label_category'].tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_array(x_train = X_train,
                                                                      y_train = y_train,
                                                                      x_test = X_test,
                                                                      y_test = y_test,
                                                                      class_names = ['0', '1', '2', '3'],
                                                                      preprocess_mode = 'bert',
                                                                      ngram_range = 1,
                                                                      maxlen = 320) # bert

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [ ]:
# define the classifier
model = text.text_classifier('bert',
                              train_data = (X_train, y_train),
                              preproc = preproc)

Is Multi-Label? False
maxlen is 320
done.


In [ ]:
learner = ktrain.get_learner(model,
                             train_data = (X_train, y_train),
                             batch_size = 2)

In [ ]:
history = learner.fit_onecycle(1e-5, 5)



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
1529/1529 [==============================] - 285s 175ms/step - loss: 1.1630 - accuracy: 0.4614
Epoch 2/5
1529/1529 [==============================] - 267s 175ms/step - loss: 0.9455 - accuracy: 0.5883
Epoch 3/5
1529/1529 [==============================] - 268s 175ms/step - loss: 0.6789 - accuracy: 0.7198
Epoch 4/5
1529/1529 [==============================] - 267s 175ms/step - loss: 0.2717 - accuracy: 0.9019
Epoch 5/5
1529/1529 [==============================] - 267s 175ms/step - loss: 0.0695 - accuracy: 0.9830


In [ ]:
learner.validate(val_data = (X_test, y_test))

11/11 [==============================] - 14s 734ms/step
              precision    recall  f1-score   support

           0       0.64      0.61      0.62        23
           1       0.64      0.73      0.68       164
           2       0.62      0.56      0.59       125
           3       0.58      0.39      0.47        28

    accuracy                           0.63       340
   macro avg       0.62      0.57      0.59       340
weighted avg       0.63      0.63      0.62       340



array([[ 14,   5,   3,   1],
       [  6, 119,  34,   5],
       [  2,  51,  70,   2],
       [  0,  11,   6,  11]])

In [ ]:
# now make predict on test file
test = pd.read_csv('/content/dev_task_b_entries.csv')
predictor = ktrain.get_predictor(learner.model, preproc)
test_result=pd.Series(predictor.predict(test_) for test_ in test['text'])

1/1 [==============================] - 0s 57ms/step


In [ ]:
label = test_result.map({'0':'1. threats, plans to harm and incitement',
                         '1':'2. derogation',
                         '2':'3. animosity',
                         '3':'4. prejudiced discussions'})

In [ ]:
test_csv= pd.DataFrame({'rewire_id':test['rewire_id'], 'label_pred':label})
test_csv.head()

,rewire_id,label_pred
0,sexism2022_english-739,3. animosity
1,sexism2022_english-10787,2. derogation
2,sexism2022_english-18547,2. derogation
3,sexism2022_english-6425,"1. threats, plans to harm and incitement"
4,sexism2022_english-10001,2. derogation


In [ ]:
test_csv['label_pred'].value_counts()

2. derogation                               248
3. animosity                                174
1. threats, plans to harm and incitement     41
4. prejudiced discussions                    23
Name: label_pred, dtype: int64

In [ ]:
test_csv.to_csv('TESTING_RESULT_TASK_B2.csv', index=False)

In [ ]:
# result in test file is 61.1 with ngram 1 and batch size 8
# result in test file is 60.2 with ngram 2 and batch size 8
# result in test file is 61.4 with ngram 2 and batch size 4 with process mode bert
# result in test file is 65 with ngram 2 and batch size 2 and 5 epoch

##test phase


In [ ]:
test_phase = pd.read_csv('/content/test_task_b_entries.csv')
predictor = ktrain.get_predictor(learner.model, preproc)
test_result=pd.Series(predictor.predict(test_) for test_ in test_phase['text'])

1/1 [==============================] - 0s 73ms/step


In [ ]:
label = test_result.map({'0':'1. threats, plans to harm and incitement',
                         '1':'2. derogation',
                         '2':'3. animosity',
                         '3':'4. prejudiced discussions'})

In [ ]:
test_csv= pd.DataFrame({'rewire_id':test_phase['rewire_id'], 'label_pred':label})
test_csv.head()

,rewire_id,label_pred
0,sexism2022_english-10731,3. animosity
1,sexism2022_english-7356,3. animosity
2,sexism2022_english-13064,4. prejudiced discussions
3,sexism2022_english-17039,3. animosity
4,sexism2022_english-14482,2. derogation


In [ ]:
test_csv['label_pred'].value_counts()

2. derogation                               504
3. animosity                                327
1. threats, plans to harm and incitement     78
4. prejudiced discussions                    61
Name: label_pred, dtype: int64

In [ ]:
test_csv.to_csv('TESTING_PHASE.csv', index=False)